In [25]:
! pip install transformers datasets
! pip install -U kaleido
!gdown 1iemxwB9aoxwRd4IUPoZTFQdD-34OpScX
!gdown 1JMrQKur7Zo0IUPrtSNYMLOz7QiNwr-Hs
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1iemxwB9aoxwRd4IUPoZTFQdD-34OpScX
To: /content/multipleChoice_data.csv
100% 179k/179k [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JMrQKur7Zo0IUPrtSNYMLOz7QiNwr-Hs
To: /content/yes_no_data.csv
100% 306k/306k [00:00<00:00, 113MB/s]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sn
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import classification_report
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch
from transformers import ElectraTokenizer, ElectraForMultipleChoice
from transformers import RobertaTokenizer, RobertaForMultipleChoice
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import XLMRobertaTokenizer, XLMRobertaForMultipleChoice
import random
from transformers import ElectraTokenizer, ElectraForMultipleChoice
from transformers import RobertaTokenizer, RobertaForMultipleChoice
from transformers import BertTokenizer, BertForMultipleChoice
from transformers import XLMRobertaTokenizer, XLMRobertaForMultipleChoice
import random
import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers import AutoModelForMultipleChoice,AutoTokenizer

plt.style.use("ggplot")
pd.set_option('display.max_colwidth', None)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [27]:
!mkdir results
!mkdir models

mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘models’: File exists


In [28]:
df = pd.read_csv("yes_no_data.csv").drop(['Unnamed: 0'],axis = 1).rename(columns = {'ans':'label'})

In [29]:
df

,choice0,choice1,question,label,context,country
0,Yes. This sentence is true.,No. This sentence is false.,"In India, it is rare for people to clean the floor with brooms.Is this sentence true?",1,broom usage,India
1,Yes. This sentence is true.,No. This sentence is false.,"In China, the driver's seat of a car is located on the right.Is this sentence true?",1,driver seat,China
2,No. This sentence is false.,Yes. This sentence is true.,"In the United States, people usually eat food with knife.Is this sentence true?",1,food and drink,United States
3,Yes. This sentence is true.,No. This sentence is false.,The falling share price is represented by the color green in China.Is this sentence true?,0,stock price,China
4,No. This sentence is false.,Yes. This sentence is true.,At traditional American funerals the color of clothing is usually red.Is this sentence true?,0,funeral,United States
...,...,...,...,...,...,...
1870,Yes. This sentence is true.,No. This sentence is false.,The most popular sports in China is badminton.Is this sentence true?,0,sports,China
1871,No. This sentence is false.,Yes. This sentence is true.,The driver's seat of a car is on the left hand side in Kenya.Is this sentence true?,0,driver seat,Kenya
1872,No. This sentence is false.,Yes. This sentence is true.,The dropping stock price will be represented by color black in Iran.Is this sentence true?,0,stock price,Iran
1873,No. This sentence is false.,Yes. This sentence is true.,At traditional Iranian funerals the color of people's clothing is usually green.Is this sentence true?,0,funeral,Iran


In [30]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [31]:
def split_data(dataset,k):
  sample_df = dataset.sample(n = k + int(k/2))
  train = sample_df.sample(n = k)
  cond1 = sample_df['question'].isin(train['question'])
  valid = sample_df.drop(sample_df[cond1].index)
  cond2 = dataset['question'].isin(sample_df['question'])
  test = dataset.drop(dataset[cond2].index)
  return train,valid,test

class QA_Tests:
  def __init__(self,k,train,test,valid,model_name,num_choice = 2):
    self.model_checkpoint = {
          'bert':"bert-base-uncased",
          'electra':"google/electra-base-discriminator",
          'roberta':"roberta-base",
          'xlm-roberta':"xlm-roberta-base",
          "bert-swag" : "ehdwns1516/bert-base-uncased_SWAG"
          }
    self.model_name = model_name
    self.model = AutoModelForMultipleChoice.from_pretrained(self.model_checkpoint[model_name])
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint[model_name], use_fast=True)
    self.k = k
    self.train,self.test,self.valid = train,test,valid
    self.num_choice = num_choice
    self.choices = []
    for i in range(0,self.num_choice):
      self.choices.append(f'choice{i}')


  def compute_metrics(self,eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

  def train_model(self,batch_size = 16, num_train_epochs=3, learning_rate=2e-5):

    self.encode_data()
    args = TrainingArguments(
    f"models/{self.model_name}-{self.k}",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    group_by_length=True,
    logging_steps=20
    )

    trainer = Trainer(
    self.model,
    args,
    train_dataset=self.encoded_train,
    eval_dataset= self.encoded_valid,
    tokenizer=self.tokenizer,
    data_collator=DataCollatorForMultipleChoice(self.tokenizer),
    compute_metrics=self.compute_metrics,
    )

    trainer.train()

  def preprocess_function(self,examples):
    question = [[context] * self.num_choice for context in examples["question"]]
    ans = [[f"{examples[choice][i]}" for choice in self.choices] for i, header in enumerate(question)]
    question = sum(question, [])
    ans = sum(ans, [])
    tokenized_examples = self.tokenizer(question, ans, truncation=True)
    return {k: [v[i:i+self.num_choice] for i in range(0, len(v), self.num_choice)] for k, v in tokenized_examples.items()}
  
  def encode_data(self):
    self.encoded_train = datasets.Dataset.from_pandas(self.train).map(self.preprocess_function, batched=True)
    self.encoded_valid = datasets.Dataset.from_pandas(self.valid).map(self.preprocess_function, batched=True)

  def predict(self):
    answers = []  
    self.model.eval()
    self.model.to(DEVICE)
    for i, row in tqdm(self.test.iterrows(), total=len(self.test)):
      q = row['question']
      choices_list = []
      for j in range(0,self.num_choice):
        choices_list.append(row[f'choice{j}'])
      encoding = self.tokenizer([q]*self.num_choice,choices_list, return_tensors="pt", padding=True)
      labels = torch.tensor(0).unsqueeze(0)
      outputs = self.model(**{k: v.unsqueeze(0).to(DEVICE) for k, v in encoding.items()}, labels=labels.to(DEVICE))
      logits = outputs.logits
      ans = torch.argmax(logits).item()
      answers.append(ans)
    return answers

## zero-shot

In [32]:
results = {'bert':[],'electra':[],'roberta':[],'xlm-roberta':[]}
train,valid,test = split_data(df,0)
for model in results.keys():
  qa = QA_Tests(0,train,test,valid,model)
  preds = qa.predict()
  results[model] = preds

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_model.bin
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifi

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_ac

  0%|          | 0/1875 [00:00<?, ?it/s]

...

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMultipleChoice: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultipleChoice were not initialized from th

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

  0%|          | 0/1875 [00:00<?, ?it/s]

In [33]:
results_df = pd.DataFrame(results)
results_df['question'] = df['question']
results_df['label'] = df['label']
results_df.to_csv('results/yes_no.csv')

In [34]:
results_df

,bert,electra,roberta,xlm-roberta,question,label
0,1,1,1,1,"In India, it is rare for people to clean the floor with brooms.Is this sentence true?",1
1,1,1,1,1,"In China, the driver's seat of a car is located on the right.Is this sentence true?",1
2,0,0,0,0,"In the United States, people usually eat food with knife.Is this sentence true?",1
3,1,1,1,1,The falling share price is represented by the color green in China.Is this sentence true?,0
4,0,0,0,0,At traditional American funerals the color of clothing is usually red.Is this sentence true?,0
...,...,...,...,...,...,...
1870,1,1,1,1,The most popular sports in China is badminton.Is this sentence true?,0
1871,0,0,0,0,The driver's seat of a car is on the left hand side in Kenya.Is this sentence true?,0
1872,0,0,0,0,The dropping stock price will be represented by color black in Iran.Is this sentence true?,0
1873,0,0,0,0,At traditional Iranian funerals the color of people's clothing is usually green.Is this sentence true?,0


In [35]:
def show_results(data):
  models = {'electra':{},'bert':{},'roberta':{},'xlm-roberta':{}}
  for name in models.keys():
  #print(model)
  #print(classification_report(data['ans'], data[name]))
    models[name]['Accuracy'] =  accuracy_score(data['label'], data[name])
    models[name]['F1 (Micro)']= f1_score(data['label'], data[name], average='micro')
    models[name]['F1 (Macro)'] = f1_score(data['label'], data[name], average='macro')
    models[name]['F1 (Weighted)'] = f1_score(data['label'], data[name], average='weighted')
    models[name]['Recall (Micro)'] = recall_score(data['label'], data[name], average='micro')
    models[name]['Recall (Macro)'] = recall_score(data['label'], data[name], average='macro')
    models[name]['Recall (Weighted)'] = recall_score(data['label'], data[name], average='weighted')
    models[name]['Precision (Micro)'] = precision_score(data['label'], data[name], average='micro')
    models[name]['Precision (Macro)'] = precision_score(data['label'], data[name], average='macro')
    models[name]['Precision (Weighted)'] = precision_score(data['label'], data[name], average='weighted')
  display(pd.DataFrame(models).transpose())

In [36]:
data = pd.read_csv('results/yes_no.csv')
show_results(data)

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.597333,0.597333,0.597168,0.597599,0.597333,0.597930,0.597333,0.597333,0.597672,0.599129
bert,0.598933,0.598933,0.598809,0.599182,0.598933,0.599676,0.598933,0.598933,0.599429,0.600915
roberta,0.578667,0.578667,0.578503,0.578942,0.578667,0.579240,0.578667,0.578667,0.579033,0.580482
xlm-roberta,0.597867,0.597867,0.597655,0.598142,0.597867,0.598324,0.597867,0.597867,0.598054,0.599484


## few-shot

In [37]:
def few_shot(k,df,num_choice = 2):
  file_name = 'yes_no'
  results = {'bert':[],'electra':[],'roberta':[],'xlm-roberta':[]}
  train,valid,test = split_data(df,k)
  for model in results.keys():
    qa = QA_Tests(k,train,test,valid,model,num_choice)
    qa.train_model()
    preds = qa.predict()
    results[model] = preds
  results_df = pd.DataFrame(results)
  results_df['question'] = df['question']
  results_df['label'] = df['label']
  if(num_choice == 3):
    file_name = 'multipleChoice'
  results_df.to_csv(f'results/{file_name}_{k}.csv')

1) k = 16

In [38]:
k = 16
few_shot(k,df)
data = pd.read_csv(f'results/yes_no_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.692798,0.750000
2,No log,0.691871,0.750000
3,No log,0.691439,0.625000


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 16
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 278044417
You're using a XLMRobertaTokenizerFast toke

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693505,0.500000
2,No log,0.688277,0.875000
3,No log,0.692114,0.500000


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward`

  0%|          | 0/1851 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.490546,0.490546,0.490374,0.490834,0.490546,0.490829,0.490546,0.490546,0.490850,0.492053
bert,0.491626,0.491626,0.491412,0.491925,0.491626,0.491806,0.491626,0.491626,0.491825,0.493030
roberta,0.508914,0.508914,0.508398,0.509181,0.508914,0.508550,0.508914,0.508914,0.508531,0.509733
xlm-roberta,0.497569,0.497569,0.497544,0.497718,0.497569,0.498428,0.497569,0.497569,0.498430,0.499636


3) k = 64

In [39]:
k = 64
few_shot(k,df)
data = pd.read_csv(f'results/yes_no_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.690661,0.645161
2,No log,0.683478,0.645161
3,No log,0.681274,0.645161


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 64
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Number of trainable parameters = 278044417
You're using a XLMRobertaTokenizerFast tok

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693813,0.354839
2,No log,0.692220,0.612903
3,No log,0.692189,0.645161


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 31
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forwar

  0%|          | 0/1763 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.498015,0.498015,0.497869,0.498330,0.498015,0.498545,0.498015,0.498015,0.498549,0.499999
bert,0.498015,0.498015,0.497869,0.498330,0.498015,0.498545,0.498015,0.498015,0.498549,0.499999
roberta,0.498015,0.498015,0.497869,0.498330,0.498015,0.498545,0.498015,0.498015,0.498549,0.499999
xlm-roberta,0.498582,0.498582,0.498283,0.498943,0.498582,0.498716,0.498582,0.498582,0.498719,0.500171


3) k = 128

In [40]:
k = 128
few_shot(k,df)
data = pd.read_csv(f'results/yes_no_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.687252,0.562500
2,No log,0.683564,0.562500
3,0.673200,0.690255,0.562500


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 128
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 278044417
You're using a XLMRobertaTokenizerFast to

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693165,0.562500
2,No log,0.693050,0.562500
3,0.695000,0.692988,0.562500


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 64
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 64
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forwar

  0%|          | 0/1659 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.493671,0.493671,0.493445,0.494006,0.493671,0.493923,0.493671,0.493671,0.493939,0.495311
bert,0.494274,0.494274,0.494061,0.494605,0.494274,0.494559,0.494274,0.494274,0.494573,0.495945
roberta,0.493671,0.493671,0.493445,0.494006,0.493671,0.493923,0.493671,0.493671,0.493939,0.495311
xlm-roberta,0.493671,0.493671,0.493445,0.494006,0.493671,0.493923,0.493671,0.493671,0.493939,0.495311


k = 256

In [41]:
k = 256
few_shot(k,df)
data = pd.read_csv(f'results/yes_no_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.711802,0.519685
2,0.696800,0.691094,0.519685
3,0.673000,0.687036,0.519685


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 256
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 48
  Number of trainable parameters = 278044417
You're using a XLMRobertaTokenizerFast to

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693069,0.519685
2,0.698900,0.692964,0.519685
3,0.692200,0.692889,0.519685


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forw

  0%|          | 0/1458 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.475995,0.475995,0.475742,0.476358,0.475995,0.476183,0.475995,0.475995,0.476249,0.477668
bert,0.475995,0.475995,0.475742,0.476358,0.475995,0.476183,0.475995,0.475995,0.476249,0.477668
roberta,0.475995,0.475995,0.475742,0.476358,0.475995,0.476183,0.475995,0.475995,0.476249,0.477668
xlm-roberta,0.475995,0.475995,0.475742,0.476358,0.475995,0.476183,0.475995,0.475995,0.476249,0.477668


In [42]:
k = 500
few_shot(k,df)
data = pd.read_csv(f'results/yes_no_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

Epoch,Training Loss,Validation Loss,Accuracy
1,0.692900,0.695757,0.528926
2,0.682600,0.691598,0.528926
3,0.654900,0.693890,0.528926


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 96
  Number of trainable parameters = 278044417
You're using a XLMRobertaTokenizerFast to

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695500,0.692908,0.528926
2,0.689000,0.691751,0.528926
3,0.679800,0.700304,0.528926


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 242
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice1, context. If __index_level_0__, question, country, choice0, choice1, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 242
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forw

  0%|          | 0/1079 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.516219,0.516219,0.515649,0.516496,0.516219,0.515811,0.516219,0.516219,0.515775,0.51706
bert,0.516219,0.516219,0.515649,0.516496,0.516219,0.515811,0.516219,0.516219,0.515775,0.51706
roberta,0.516219,0.516219,0.515649,0.516496,0.516219,0.515811,0.516219,0.516219,0.515775,0.51706
xlm-roberta,0.516219,0.516219,0.515649,0.516496,0.516219,0.515811,0.516219,0.516219,0.515775,0.51706


# Mutiple Choice

zero-shot

In [43]:
results = {'bert':[],'electra':[],'roberta':[],'xlm-roberta':[]}
df_mc = pd.read_csv('multipleChoice_data.csv').rename(columns = {'ans':'label'})
train,valid,test = split_data(df_mc,0)
for model in results.keys():
  qa = QA_Tests(0,train,test,valid,model,3)
  preds = qa.predict()
  results[model] = preds

results_df = pd.DataFrame(results)
results_df['question'] = df_mc['question']
results_df['label'] = df_mc['label']
results_df.to_csv('results/multipleChoice.csv')
data = pd.read_csv('results/multipleChoice.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--electra-base-discriminator/snapshots/1b48ef100dac4676d84125a8a7b7ab7c51e00386/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-base-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

  0%|          | 0/1875 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.335467,0.335467,0.335501,0.335407,0.335467,0.335795,0.335467,0.335467,0.335457,0.335597
bert,0.241600,0.241600,0.241647,0.241508,0.241600,0.242050,0.241600,0.241600,0.241774,0.241956
roberta,0.354667,0.354667,0.354601,0.354815,0.354667,0.354901,0.354667,0.354667,0.355007,0.355685
xlm-roberta,0.296533,0.296533,0.296428,0.296614,0.296533,0.296478,0.296533,0.296533,0.296518,0.296833


few-shot

k = 16

In [44]:
k = 16
few_shot(k,df_mc,3)
data = pd.read_csv(f'results/multipleChoice_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.wa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.081365,0.250000
2,No log,1.083831,0.375000
3,No log,1.082979,0.375000


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 16
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 278044417
Y

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.099173,0.250000
2,No log,1.099135,0.250000
3,No log,1.099069,0.250000


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in the evaluat

  0%|          | 0/1715 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.317784,0.317784,0.317746,0.318038,0.317784,0.317652,0.317784,0.317784,0.318049,0.318496
bert,0.312536,0.312536,0.312331,0.312521,0.312536,0.312365,0.312536,0.312536,0.312306,0.312515
roberta,0.326531,0.326531,0.326461,0.326980,0.326531,0.326338,0.326531,0.326531,0.327152,0.327994
xlm-roberta,0.325364,0.325364,0.325228,0.325540,0.325364,0.325295,0.325364,0.325364,0.325508,0.326065


k = 64

In [45]:
k = 64
few_shot(k,df_mc,3)
data = pd.read_csv(f'results/multipleChoice_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.wa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.115545,0.227273
2,No log,1.102803,0.227273
3,No log,1.100692,0.318182


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 64
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Number of trainable parameters = 278044417


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.091542,0.363636
2,No log,1.101843,0.363636
3,No log,1.098976,0.318182


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 22
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 22
  Batch size = 16
The following columns in the evalu

  0%|          | 0/1372 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.325802,0.325802,0.325773,0.325849,0.325802,0.326194,0.325802,0.325802,0.326065,0.326617
bert,0.357872,0.357872,0.357384,0.358062,0.357872,0.357387,0.357872,0.357872,0.357488,0.358360
roberta,0.321429,0.321429,0.321156,0.321791,0.321429,0.321155,0.321429,0.321429,0.321574,0.322577
xlm-roberta,0.305394,0.305394,0.305242,0.305567,0.305394,0.305544,0.305394,0.305394,0.305744,0.306550


k = 128

In [46]:
k = 128
few_shot(k,df_mc,3)
data = pd.read_csv(f'results/multipleChoice_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.wa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.077570,0.487805
2,No log,1.055023,0.585366
3,1.074800,1.039558,0.585366


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 128
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 278044417

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.099153,0.341463
2,No log,1.095971,0.439024
3,1.109200,1.095912,0.414634


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 41
  Batch size = 16
The following columns in the evalu

  0%|          | 0/1111 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.322232,0.322232,0.322343,0.322898,0.322232,0.322260,0.322232,0.322232,0.323471,0.324649
bert,0.319532,0.319532,0.318841,0.320050,0.319532,0.318753,0.319532,0.319532,0.319328,0.320972
roberta,0.338434,0.338434,0.337393,0.338763,0.338434,0.337499,0.338434,0.338434,0.337843,0.339636
xlm-roberta,0.323132,0.323132,0.322995,0.323109,0.323132,0.323718,0.323132,0.323132,0.323501,0.324299


k = 256

In [47]:
k = 256
few_shot(k,df_mc,3)
data = pd.read_csv(f'results/multipleChoice_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.wa

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.050193,0.441558
2,1.073200,0.989395,0.493506
3,1.011400,0.966338,0.545455


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 256
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 48
  Number of trainable parameters = 278044417

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.097859,0.415584
2,1.104300,1.097608,0.441558
3,1.095900,1.097453,0.467532


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 77
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 77
  Batch size = 16
The following columns in the evalu

  0%|          | 0/776 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.324742,0.324742,0.324122,0.325200,0.324742,0.324444,0.324742,0.324742,0.324794,0.326626
bert,0.327320,0.327320,0.326629,0.327865,0.327320,0.326736,0.327320,0.327320,0.327155,0.329037
roberta,0.331186,0.331186,0.330033,0.331328,0.331186,0.330139,0.331186,0.331186,0.330082,0.331632
xlm-roberta,0.324742,0.324742,0.323949,0.324920,0.324742,0.324506,0.324742,0.324742,0.324558,0.326314


In [48]:
k = 500
few_shot(k,df_mc,3)
data = pd.read_csv(f'results/multipleChoice_{k}.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.wa

Epoch,Training Loss,Validation Loss,Accuracy
1,1.110400,1.009114,0.756522
2,0.940800,0.649624,0.808696
3,0.821700,0.561450,0.808696


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 96
  Number of trainable parameters = 278044417

Epoch,Training Loss,Validation Loss,Accuracy
1,1.088300,1.098078,0.504348
2,1.098400,1.097461,0.573913
3,1.097300,1.096924,0.626087


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 115
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMultipleChoice.forward` and have been ignored: __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context. If __index_level_0__, question, country, choice0, choice2, choice1, Unnamed: 0, context are not expected by `XLMRobertaForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 115
  Batch size = 16
The following columns in the eva

  0%|          | 0/395 [00:00<?, ?it/s]

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.362025,0.362025,0.360004,0.363005,0.362025,0.360106,0.362025,0.362025,0.360872,0.364985
bert,0.372152,0.372152,0.370059,0.373223,0.372152,0.370001,0.372152,0.372152,0.371485,0.375652
roberta,0.379747,0.379747,0.374622,0.378882,0.379747,0.374820,0.379747,0.379747,0.374675,0.378283
xlm-roberta,0.346835,0.346835,0.346503,0.347152,0.346835,0.348102,0.346835,0.346835,0.347752,0.350459


In [49]:
def show_swg_results(data):
  models = {"bert-swag":{}}
  for name in models.keys():
    #print(model)
    #print(classification_report(data['ans'], data[name]))
    models[name]['Accuracy'] =  accuracy_score(data['label'], data[name])
    models[name]['F1 (Micro)']= f1_score(data['label'], data[name], average='micro')
    models[name]['F1 (Macro)'] = f1_score(data['label'], data[name], average='macro')
    models[name]['F1 (Weighted)'] = f1_score(data['label'], data[name], average='weighted')
    models[name]['Recall (Micro)'] = recall_score(data['label'], data[name], average='micro')
    models[name]['Recall (Macro)'] = recall_score(data['label'], data[name], average='macro')
    models[name]['Recall (Weighted)'] = recall_score(data['label'], data[name], average='weighted')
    models[name]['Precision (Micro)'] = precision_score(data['label'], data[name], average='micro')
    models[name]['Precision (Macro)'] = precision_score(data['label'], data[name], average='macro')
    models[name]['Precision (Weighted)'] = precision_score(data['label'], data[name], average='weighted')
    display(pd.DataFrame(models).transpose())


def few_shot_swg(k,df):
  results = {"bert-swag":[]}
  train,valid,test = split_data(df,k)
  for model in results.keys():
    qa = QA_Tests(k,train,test,valid,model,4)
    qa.train_model()
    preds = qa.predict()
    results[model] = preds

  results_df = pd.DataFrame(results)
  results_df['question'] = df['question']
  results_df['label'] = df['label']
  results_df.to_csv(f'results/multipleChoiceSWAG_{k}.csv')
  data = pd.read_csv(f'results/multipleChoiceSWAG_{k}.csv')
  show_swg_results(data)


In [50]:
results = {"bert-swag":[]}
df_mc_4 = pd.read_csv('multipleChoice_data.csv').rename(columns = {'ans':'label'})
df_mc_4['choice3'] = 'none'
train,valid,test = split_data(df_mc_4,0)
for model in results.keys():
  qa = QA_Tests(0,train,test,valid,model,4)
  preds = qa.predict()
  results[model] = preds

results_df = pd.DataFrame(results)
results_df['question'] = df_mc_4['question']
results_df['label'] = df_mc_4['label']
results_df.to_csv('results/multipleChoiceSWAG.csv')
data = pd.read_csv('results/multipleChoiceSWAG.csv')
show_swg_results(data)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ehdwns1516--bert-base-uncased_SWAG/snapshots/41f1703c5e32cea8ad48180e4520184daa9677a7/config.json
Model config BertConfig {
  "_name_or_path": "ehdwns1516/bert-base-uncased_SWAG",
  "architectures": [
    "BertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ehdwns1516-

  0%|          | 0/1875 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,F1 (Macro),F1 (Micro),F1 (Weighted),Precision (Macro),Precision (Micro),Precision (Weighted),Recall (Macro),Recall (Micro),Recall (Weighted)
bert-swag,0.414933,0.324593,0.414933,0.433452,0.339733,0.414933,0.453704,0.310746,0.414933,0.414933


In [51]:
few_shot_swg(16,df_mc_4)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ehdwns1516--bert-base-uncased_SWAG/snapshots/41f1703c5e32cea8ad48180e4520184daa9677a7/config.json
Model config BertConfig {
  "_name_or_path": "ehdwns1516/bert-base-uncased_SWAG",
  "architectures": [
    "BertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ehdwns1516-

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this war

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.170356,0.375000
2,No log,3.022318,0.375000
3,No log,2.955002,0.375000


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 16
The following columns in

  0%|          | 0/1715 [00:00<?, ?it/s]

,Accuracy,F1 (Macro),F1 (Micro),F1 (Weighted),Precision (Macro),Precision (Micro),Precision (Weighted),Recall (Macro),Recall (Micro),Recall (Weighted)
bert-swag,0.337609,0.33783,0.337609,0.33742,0.337757,0.337609,0.337526,0.338207,0.337609,0.337609


In [52]:
few_shot_swg(500,df_mc_4)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ehdwns1516--bert-base-uncased_SWAG/snapshots/41f1703c5e32cea8ad48180e4520184daa9677a7/config.json
Model config BertConfig {
  "_name_or_path": "ehdwns1516/bert-base-uncased_SWAG",
  "architectures": [
    "BertForMultipleChoice"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ehdwns1516-

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this war

Epoch,Training Loss,Validation Loss,Accuracy
1,1.825700,0.464496,0.817391
2,0.678300,0.384534,0.826087
3,0.677700,0.352687,0.826087


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 115
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context. If choice3, __index_level_0__, question, choice0, choice2, country, choice1, Unnamed: 0, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 115
  Batch size = 16
The following column

  0%|          | 0/395 [00:00<?, ?it/s]

,Accuracy,F1 (Macro),F1 (Micro),F1 (Weighted),Precision (Macro),Precision (Micro),Precision (Weighted),Recall (Macro),Recall (Micro),Recall (Weighted)
bert-swag,0.359494,0.356312,0.359494,0.360476,0.357459,0.359494,0.362757,0.35645,0.359494,0.359494


In [60]:
results = {'bert':[],'electra':[],'roberta':[],'xlm-roberta':[]}

train,valid,test = split_data(df_mc_4,0)
for model in results.keys():
  qa = QA_Tests(0,train,test,valid,model,4)
  preds = qa.predict()
  results[model] = preds

results_df = pd.DataFrame(results)
results_df['question'] = df_mc_4['question']
results_df['label'] = df_mc_4['label']
results_df.to_csv('results/multipleChoice_4swag.csv')
data = pd.read_csv('results/multipleChoice_4swag.csv')
show_results(data)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_m

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--electra-base-discriminator/snapshots/1b48ef100dac4676d84125a8a7b7ab7c51e00386/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-base-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_

  0%|          | 0/1875 [00:00<?, ?it/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

  0%|          | 0/1875 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and 

,Accuracy,F1 (Micro),F1 (Macro),F1 (Weighted),Recall (Micro),Recall (Macro),Recall (Weighted),Precision (Micro),Precision (Macro),Precision (Weighted)
electra,0.220267,0.220267,0.174650,0.233467,0.220267,0.164922,0.220267,0.220267,0.185815,0.248648
bert,0.162667,0.162667,0.147414,0.196195,0.162667,0.122317,0.162667,0.162667,0.186110,0.247973
roberta,0.107733,0.107733,0.102578,0.136617,0.107733,0.080956,0.107733,0.107733,0.140276,0.187099
xlm-roberta,0.081067,0.081067,0.097171,0.129763,0.081067,0.060722,0.081067,0.081067,0.243342,0.325333
